In [ ]:
import pandas as pd
import ast
df_rating = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\ratings.csv")
df_rating1 = df_rating.sample(n=3000, replace=False, random_state=1)
df_mv = pd.read_csv(r"C:\Users\sjlal\NYP\Project\FinalData\movies_metadata.csv")
# generate mv features
df_mv['id'] = df_mv['id'].astype(str)
df_rating['movieId'] = df_rating['movieId'].astype(str)
df_rate_mv_pro = pd.merge(df_mv, df_rating, how='right', left_on=['id'], right_on=['movieId'])
def get_genre(outterlist):
    outterlist = ast.literal_eval(outterlist)
    lst = []
    for a in outterlist:
        try:
            lst.append(a['name'])
        except IndexError:
            lst.append('')
    return '|'.join(lst)

df_rate_mv_pro1 = df_rate_mv_pro[['genres','id','title','userId','movieId', 'rating']]
df_rate_mv_pro1.dropna(inplace=True)
df_rate_mv_pro1['genres_descp'] = df_rate_mv_pro1['genres'].apply(get_genre)
df_mv3 = df_rate_mv_pro1.loc[~df_rate_mv_pro1["id"].str.contains('-')]
df_mv3.drop_duplicates(subset =['id', 'title'], 
                     keep = 'first', inplace = True)
df_mv3 = df_mv3.loc[df_mv3['genres_descp'].str.len()>3]
df_mv4 = df_mv3.sample(n=3000, replace=False, random_state=1)
df_mv4.reset_index(inplace=True)
df_user_le = pd.DataFrame(df_mv4["userId"].tolist(), columns=['user'])
le = preprocessing.LabelEncoder()
df_user_le['user_le'] = le.fit_transform(df_user_le['user'])
df_mv_le = pd.DataFrame(df_mv4["movieId"].tolist(), columns=['movie'])
df_mv_le['mv_le'] = le.fit_transform(df_mv_le['movie'])

In [ ]:
user_ids = np.array(df_user_le['user_le'].tolist())
movie_ids = np.array(df_mv_le['mv_le'].tolist())
user_ratings = np.array(df_mv4["rating"].tolist())
print ('num of users:',df_mv4['userId'].nunique())
print ('num of movies:',df_mv4['movieId'].nunique())

In [ ]:
graph = tf.Graph()
n_movie = 3000
n_user = 1105
embedding_size = 30

lr = 0.0001
reg = 0.01

with graph.as_default():
    user = tf.placeholder(tf.int32, name="user_id") 
    movie = tf.placeholder(tf.int32, name="movie_id") 
    rating = tf.placeholder(tf.float32, name="rating") 

    movie_embedding = tf.Variable(tf.truncated_normal([n_movie, embedding_size], stddev=0.02, mean=0.) ,name="movie_embedding")
    user_embedding = tf.Variable(tf.truncated_normal([n_user, embedding_size], stddev=0.02, mean=0.) ,name="user_embedding")
    
    movie_bias_embedding = tf.Variable(tf.truncated_normal([n_movie], stddev=0.02, mean=0.) ,name="movie_bias_embedding")
    user_bias_embedding = tf.Variable(tf.truncated_normal([n_user], stddev=0.02, mean=0.) ,name="user_bias_embedding")
    
    
    global_bias = tf.Variable(tf.truncated_normal([], stddev=0.02, mean=0.) ,name="global_bias")
    
    u = tf.nn.embedding_lookup(user_embedding, user)
    m = tf.nn.embedding_lookup(movie_embedding, movie)
    
    u_bias = tf.nn.embedding_lookup(user_bias_embedding, user)
    m_bias = tf.nn.embedding_lookup(movie_bias_embedding, movie)
    

    predicted_rating = tf.reduce_sum(tf.multiply(u, m), 1) + u_bias + m_bias + global_bias

    rmse = tf.sqrt(tf.reduce_mean(tf.square(predicted_rating - rating))) # RMSE
    cost = tf.nn.l2_loss(predicted_rating - rating)
    regularization = reg * (tf.nn.l2_loss(movie_embedding) + tf.nn.l2_loss(user_embedding)
                            + tf.nn.l2_loss(movie_bias_embedding) + tf.nn.l2_loss(user_bias_embedding))
    
    loss = cost + regularization
    
    optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [ ]:
batch_size = 5
n_epoch = 30


with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    for _ in range(n_epoch):
        for start in range(0, user_ratings.shape[0] - batch_size, batch_size):
            end = start + batch_size
            _, cost_value = sess.run([optimizer, rmse], feed_dict={user: user_ids[start:end],
                                                  movie: movie_ids[start: end],
                                                  rating: user_ratings[start: end]})

        print ("RMSE", cost_value)
    embeddings = movie_embedding.eval()